# 2. Study Area Geometry
This notebook sets up the geometry for a study area. The geometry might be used
in a later notebook (`03-Gamma-Conjugate-Prior-Parameters.ipynb`) to exclude heat
flow data from the study area from the conjugate prior parameter estimation. This
in turn ensures independence of the prior data from the later analysis.

Package imports:

In [ ]:
import numpy as np
from pyproj import Proj
from pickle import Pickler
from shapely.geometry import Polygon as SPolygon
from flottekarte import Map
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MPolygon
from zeal2022hf import write_polygon_geojson, read_geopackage_polys

Configure plots to look good on a HiDPI monitor (you may not need the following configuration if you are not using a HiDPI monitor):

In [ ]:
%config InlineBackend.figure_format = 'retina'

## Setting up the Coordinate System:
A PROJ string to work in later (might also work with WKT):

In [ ]:
proj_str = "+proj=omerc +lat_0=64.6978215 +lonc=-169.71419431 +alpha=-71.26503909 +gamma=-30.62172945 " \
           "+k=0.99959372 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs"
proj = Proj(proj_str)

## Define the Polygons of Data Selection
Load a number of polygons whose union defines the joint selection area of regional
heat flow data used in the later analysis steps (i.e. one regional aggregate distribution
per polygon).

Here, we load polygons for the analysis in Southern California stored in a GeoPackage format.
The following field can be adjusted to any set of polygons as long as `selection_polys` contains
a list `lola` of NumPy arrays of shape $(N_i,2)$, where $N_i$ is the number of data points of the
array `lola[i]`. Furthermore, `lola[i][:,0]` should refer to the longitudes and `lola[i][:,1]` to
the latitudes of the Polygon's coordinates.

In [ ]:
selection_polys = read_geopackage_polys("data/SoCal-Regions.gpkg", proj_str)

Now project the polygons to the working coordinate system:

In [ ]:
selection_polys_xy = [np.array(proj(*lola.T)).T for lola in selection_polys]

##### Additional Polygons
Determine the region of interest used in the ENCOS paper (Ziebarth *et al.*, 2020).

This step might not be needed for the analysis in other regions - or you might want
to load and project other geometries here.

In [ ]:
# As defined in Ziebarth et al. (2020):
proj_str_encos = "proj=omerc lonc=-117.60812863592994 lat_0=34.25571056443553 alpha=-53.56972535964464 " \
                 "k_0=0.9999877834134252 +no_rot +no_off"

# Bounds in this projection (including u offset u_0):
xlim_encos = (-349375.08186776284, 336921.92762642726)
ylim_encos = (-345061.06198712916, 245061.0620845298)

# Correct for the u_0 offset:
u_0 = Proj(proj_str_encos)(-117.60812863592994, 34.25571056443553)[0]
xlim_encos = (xlim_encos[0]+u_0, xlim_encos[1]+u_0)
ylim_encos = (-ylim_encos[1], -ylim_encos[0])

# Now in this system, we can invert the SoCal ENCOS area bounds:
proj_encos = Proj(proj_str_encos)
ENCOS_inner_extent_lola = proj_encos([*xlim_encos, *xlim_encos[::-1]],
                                     [ylim_encos[0],ylim_encos[0], ylim_encos[1], ylim_encos[1]],
                                     inverse=True)

# Also define a more fine-grained polygon since straight lines in the
# ENCOS coordinate system need not be straight lines in this working system:
xpoly_encos = np.concatenate([np.linspace(*xlim_encos, 100), np.ones(98)*xlim_encos[1],
                              np.linspace(*xlim_encos[::-1],100),np.ones(98)*xlim_encos[0]])
ypoly_encos = np.concatenate([np.ones(100)*ylim_encos[0], np.linspace(*ylim_encos, 100)[1:-1],
                              np.ones(100)*ylim_encos[1], np.linspace(*ylim_encos[::-1],100)[1:-1]])
ENCOS_poly_lola = np.array(proj_encos(xpoly_encos, ypoly_encos, inverse=True)).T
ENCOS_poly_xy = np.array(proj(*ENCOS_poly_lola.T)).T

## Geometry Statistics and Visualization
Here, plot the geometry in the working coordinate system to ensure that everythin worked out as it should.
Also, we output some statistics about the data.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
mp = Map.for_data(np.concatenate([ENCOS_poly_lola[:,0], *[xy[:,0] for xy in selection_polys]]),
                  np.concatenate([ENCOS_poly_lola[:,1], *[xy[:,1] for xy in selection_polys]]),
                  proj_str, ax)

ax.add_patch(MPolygon(np.array(proj(*ENCOS_poly_lola.T)).T, facecolor='none', edgecolor='tab:blue'))

for poly in selection_polys:
    ax.add_patch(MPolygon(np.array(proj(*poly.T)).T, facecolor='none', edgecolor='tab:orange'))

ax.scatter(*proj(*ENCOS_inner_extent_lola))

mp.plot_axes()

In [ ]:
for poly in selection_polys:
    print("A =",SPolygon(np.array(proj(*poly.T)).T).area * 1e-6, "km²")
    print("  -> R =",
          1e-3*np.sqrt(SPolygon(np.array(proj(*poly.T)).T).area / np.pi), 'km')

## Data Export
Here, export geometry information. The important parts for the following analysis notebooks
are the `"proj_str"`, `"selection_polygons_lola"`, and `"selection_polygons_xy"` keys. Those
should also be present when analyzing other study areas.

In [ ]:
geometry = {
    "proj_str" : proj_str,
    "selection_polygons_lola" : selection_polys,
    "selection_polygons_xy" : selection_polys_xy,
    "encos_poly_lola" : ENCOS_poly_lola,
    "encos_poly_xy" : ENCOS_poly_xy
}

with open('export/06a-SoCal-Geometry.pickle','wb') as f:
    Pickler(f).dump(geometry)

Output the ENCOS polygon:

In [ ]:
write_polygon_geojson('export/ENCOS-area.geojson',
                      ENCOS_poly_lola[:,0], ENCOS_poly_lola[:,1],
                      [], fields=[], name='ENCOS area')

### References
>  Ziebarth, M. J., von Specht, S., Heidbach, O., Cotton, F., & Anderson, J. G. (2020).
>      Applying conservation of energy to estimate earthquake frequencies from strain rates and stresses.
>      Journal of Geophysical Research: Solid Earth, 125, e2020JB020186.
>      https://doi.org/10.1029/2020JB020186 

### License
```
A notebook to define the study area geometry in a 2D map projection system.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```